In [1]:
var = 'error'
lons = np.arange(-180,180,10)
lats = np.arange(-90,90,10)
fig,ax = plt.subplots(figsize=(10,7),ncols=2)
for i in lons:
    for j in lats:
        prof_rho2 = ds_rho2.sel(xh=i,yh=j,method='nearest').squeeze()
        if np.where(~np.isnan(prof_rho2[var]))[0].size != 0:
            z = prof_rho2.thkcello.cumsum(dim='rho2_l').where(prof_rho2.thkcello!=0,drop=True)
            v = prof_rho2[var].where(prof_rho2.thkcello!=0,drop=True)
            ax[0].plot(v,z,color='lightgray',marker='.')
            
        prof_z = ds_z.sel(xh=i,yh=j,method='nearest').squeeze()
        if np.where(~np.isnan(prof_z[var]))[0].size != 0:
            z = prof_z.thkcello.cumsum(dim='z_l').where(prof_z.thkcello!=0,drop=True)
            v = prof_z[var].where(prof_z.thkcello!=0,drop=True)
            ax[1].plot(v,z,color='lightgray',marker='.')
ax[0].invert_yaxis()
ax[0].grid()
ax[1].invert_yaxis()
ax[1].grid()

NameError: name 'np' is not defined

In [ ]:
######
# PATCHING OF AN ERROR IN XHISTOGRAM - ALLOWS XARRAY DATAARRAYS TO BE HISTOGRAMMED FROM THE CORE ALGORITHM
######
def histsum_ds(ds,bin_data,bins,dim,verbose=False,variables=None):
    ds = ds.copy()
    vol = ds['volcello']
    if variables is not None:
        ds = ds[variables]
    ds_rebinned = xr.Dataset(coords={'time':np.arange(0,12,1),bin_data.name+'_bin':0.5*(bins[1:]+bins[:-1])})
    for var in ds.data_vars:
        if verbose:
            print(var)
            var
        if ds[var].dtype == 'float32':
            nanmask = np.isnan(ds[var])
            ds_rebinned[var] = (['time',bin_data.name+'_bin'],histogram(
                bin_data.where(~nanmask), bins=[bins], axis=dim, weights=(ds[var]*vol).where(~nanmask),block_size=None
            ))
    return ds_rebinned